In [ ]:
import mlrun
from mlrun.errors import MLRunRuntimeError
from mlrun.runtimes import RunError
import os

In [ ]:
project_name1 = "test-git-flow"
project = mlrun.get_or_create_project(project_name1, "./test-git-flow")

## Check set_source does change the project spec

In [ ]:
source_path = "git://github.com/mlrun/project-demo#main"
project.set_source(source_path, pull_at_runtime=True)
assert project.spec.source==source_path

## Check the auto_build parameter + check that pull_at_runtime=True works
(There's no way of really checking if pull_at_runtime works like it should during automation tests.. So I'll check that it works with both values)

In [ ]:
handler_name = "gen_iris.iris_generator"
func1_name = "func1-name"
func1 = project.set_function(handler=handler_name, name=func1_name, image="mlrun/mlrun", kind="job", with_repo=True)

### Run the function with auto_build=False
#### We expect to receive an MLRunRuntimeError

In [ ]:
try:
    func1_run = project.run_function(func1_name, local=False, auto_build=False)
except Exception as e:
    assert isinstance(e, MLRunRuntimeError)

### Now we set auto_build=True
#### We expect the function to run successfully

In [ ]:
func1_run = project.run_function(func1_name, local=False, auto_build=True)
assert isinstance(func1_run, mlrun.model.RunObject)
assert func1_run.spec.handler==handler_name

## Check the with_repo parameter + check that pull_at_runtime=False works
### We will create a new project to check pull_at_runtime=False


In [ ]:
project_name2 = "test-git-flow2"
project = mlrun.get_or_create_project(project_name2, "./test-git-flow", allow_cross_project=True)

In [ ]:
project.set_source("git://github.com/mlrun/project-demo#main", pull_at_runtime=False)

#### First we try with_repo=False. Since source is from github we expect an error

In [ ]:
handler_name = "gen_iris.iris_generator"
func2_name = "func2-name"
func2 = project.set_function(handler=handler_name, name=func2_name, image="mlrun/mlrun", kind="job", with_repo=False)

In [ ]:
try:
    func2_run = project.run_function(func2_name, local=False, auto_build=True)
except Exception as e:
    assert isinstance(e, RunError)

#### Now we set with_repo=True - we expect to succeed

In [ ]:
func2 = project.set_function(handler=handler_name, name=func2_name, image="mlrun/mlrun", kind="job", with_repo=True)

In [ ]:
func2_run = project.run_function(func2_name, local=False, auto_build=True)
assert isinstance(func2_run, mlrun.model.RunObject)
assert func2_run.spec.handler==handler_name

## Check the 'create_remote' function

In [ ]:
project_name3 = "test-git-flow3"
project = mlrun.get_or_create_project(project_name3, "./test-git-flow", init_git=True, allow_cross_project=True)


In [ ]:
source_path = 'git://github.com/mlrun/project-demo'
project.create_remote(url=source_path,name='test-git-flow-remote',branch='main')
assert project.spec.source==source_path+"#refs/heads/master"
assert project.spec.origin_url==source_path+"#main"

### Check the 'load_project' function

In [ ]:
source_path = 'git://github.com/mlrun/project-demo'
folder_name = "loaded_project"
project = mlrun.load_project(folder_name, source_path, clone=True)


In [ ]:
folder_name = "loaded_project"
created_files = ["gen_iris.py", "kflow.py", "LICENSE", "newflow.py", "prep_data.py", "project.yaml", "README.md"]

assert os.path.exists(folder_name), f"The file '{folder_name}' does not exist."
for file_name in created_files:
    file_path = folder_name+"/"+file_name
    assert os.path.exists(file_path), f"The file '{file_path}' does not exist."

## Check the 'export' function

In [ ]:
export_file_name = "export-file.yaml"
project.export(export_file_name)
assert os.path.exists(export_file_name), f"The file '{export_file_name}' does not exist."


## Delete all the resources

In [ ]:
run_db = mlrun.get_run_db()

In [ ]:
run_db.delete_project(project_name1, mlrun.common.schemas.constants.DeletionStrategy.cascade)
run_db.delete_project(project_name2, mlrun.common.schemas.constants.DeletionStrategy.cascade)
run_db.delete_project(project_name3, mlrun.common.schemas.constants.DeletionStrategy.cascade)